Data: https://universe.roboflow.com/utwente/crime-data-labelling/browse?queryText=class%3Astealing&pageSize=200&startingIndex=0&browseQuery=true

In [1]:
!pip install ultralytics
!pip -q install --upgrade ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 19.6 MB/s eta 0:00:00


In [2]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="APIKI")
#project = rf.workspace("armed-robbery-aixzv").project("armed-robbery")
project = rf.workspace("projetosolues").project("crime-data-labelling-ppz9h")
version = project.version(1)
dataset = version.download("yolov11")




   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.7/88.7 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 46.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 68.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 132.4 MB/s eta 0:00:00
  Attempting uninstall: opencv-python-headless
    Found existing installation: opencv-python-headless 4.12.0.88
    Uninstalling opencv-python-headless-4.12.0.88:
      Successfully uninstalled opencv-python-headless-4.12.0.88
  Attempting uninstall: idna
    Found existing installation: idna 3.10
    Uninstalling idna-3.10:
      Successfully uninstalled idna-3.10
loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to crime-data-labelling-1 in yolov11:: 100%|██████████| 2600/2600 [00:00<00:00, 11081.54it/s]


In [ ]:
from ultralytics import YOLO
import os

DATA_YAML  = "/content/crime-data-labelling-1/data.yaml"
RESULTS_DIR = "/content/results_medium"
MODEL = "yolo11m.pt"
RUN_NAME = "robbery_y11m_960_medium"

os.makedirs(RESULTS_DIR, exist_ok=True)

model = YOLO(MODEL)

results = model.train(
    data=DATA_YAML,
    epochs=80,
    imgsz=960,
    batch=-1,
    cache=True,
    close_mosaic=10,
    project=RESULTS_DIR,
    name=RUN_NAME,
    exist_ok=True,
)

run_dir = f"{RESULTS_DIR}/{RUN_NAME}"
print("Run dir:", run_dir)

In [ ]:
from IPython.display import Image, display
import os

for f in ["results.png", "PR_curve.png", "confusion_matrix.png"]:
    p = os.path.join(run_dir, f)
    if os.path.exists(p):
        display(Image(filename=p))

In [ ]:
torch.save(model.model, "crime-detector.pt")

In [ ]:
from ultralytics import YOLO
import os.path as osp, torch

# --- CONFIG ---
RESULTS_DIR = "/content/results_medium"
RUN_NAME    = "robbery_y11m_960_medium"
VIDEO_PATH  = "/content/a.MOV"
IMG_SIZE    = 960
CONF        = 0.35
IOU         = 0.5

# --- Pesos ---
best_ckpt = osp.join(RESULTS_DIR, RUN_NAME, "weights", "best.pt")
model = YOLO(best_ckpt)

# --- Evaluacion --
out_name = f"{RUN_NAME}_infer"

gen = model.predict(
    source=VIDEO_PATH,
    imgsz=IMG_SIZE,
    conf=CONF,
    iou=IOU,
    device=0 if torch.cuda.is_available() else "cpu",
    save=True,
    stream=True,
    vid_stride=1,
    project=RESULTS_DIR,
    name=out_name,
    exist_ok=True
)
for _ in gen:  # consume el generador sin acumular resultados
    pass


# ---  AVI to MP4 ---
pred_dir = osp.join(RESULTS_DIR, out_name)
avi_path = [f for f in os.listdir(pred_dir) if f.endswith(".avi")]
avi_path = osp.join(pred_dir, avi_path[0])

mp4_path = avi_path.replace(".avi", ".mp4")
cmd = [
    "ffmpeg","-y","-i", avi_path,
    "-c:v","libx264","-preset","fast","-crf","23",
    "-c:a","aac","-b:a","128k",
    "-movflags","+faststart",
    mp4_path
]
subprocess.run(cmd, check=True)

print("Video final en MP4:", mp4_path)